<a href="https://colab.research.google.com/github/pragnacodes/tweet-sentiment/blob/main/Tweet_Sentiment_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [4]:
twitter_data = pd.read_csv(r"https://drive.google.com/file/d/14FPgHupIj8l8BjIqOrT4f3V7YS29saGN/view?usp=drive_link", encoding='ISO-8859-1')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 3433


In [9]:
# Step 1: Install gdown if not already installed
!pip install -U gdown

# Step 2: Import gdown
import gdown

# Step 3: Use the correct file ID
file_id = "1AZbR5a9o_vtNPeeYs9fybukzCHTgqtjc"  # From your Google Drive URL
url = f"https://drive.google.com/uc?id={file_id}"

# Step 4: Download the file using gdown
output = "twitter_data.csv"
gdown.download(url, output, quiet=False)

# Step 5: Load the downloaded CSV file using pandas
import pandas as pd
twitter_data = pd.read_csv(output, encoding='ISO-8859-1')  # Or encoding='utf-8' if needed
twitter_data.head()


Downloading...
From (original): https://drive.google.com/uc?id=1AZbR5a9o_vtNPeeYs9fybukzCHTgqtjc
From (redirected): https://drive.google.com/uc?id=1AZbR5a9o_vtNPeeYs9fybukzCHTgqtjc&confirm=t&uuid=f839f512-fad4-44ab-ba95-ba72760195c7
To: /content/twitter_data.csv
100%|██████████| 239M/239M [00:01<00:00, 121MB/s]


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
!pip install -U gdown
import gdown


In [6]:
twitter_data = pd.read_csv("/content/tweet.csv", encoding='cp1252')        # Common for Windows


FileNotFoundError: [Errno 2] No such file or directory: '/content/tweet.csv'

In [10]:
twitter_data.shape

(1599999, 6)

In [13]:
column_names= ['target', 'id', 'date','flag', 'user', 'text']
twitter_data = pd.read_csv(output, names= column_names, encoding='ISO-8859-1')

In [15]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
#Checking the total no of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [17]:
#checking the distribution of the target column
twitter_data["target"].value_counts()

,count
target,
0,800000
4,800000


In [18]:
#Convert the target "4" to "1"
twitter_data.replace({'target':{4:1}}, inplace = True)

In [19]:
twitter_data["target"].value_counts()

,count
target,
0,800000
1,800000


'0' is negative and '1' is positive

**STEMMING**

In [20]:
port_stem= PorterStemmer() # instance of PorterStemmer in port_stem

In [21]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

  return stemmed_content

In [ ]:
twitter_data['stemmed_content']= twitter_data['text'].apply(stemming)


In [ ]:
print(twitter_data["stemmed_content"])

NameError: name 'twitter_data' is not defined

In [ ]:
print(twitter_data["target"])

In [ ]:
#separating the data and the label
X= twitter_data["stemmed_content"].values
Y= twitter_data["target"].values

In [ ]:
#Splitting the data into test and train
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X_train)

In [ ]:
#converting the textual value to numerical value
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)
print(X_test)

In [ ]:
#Training the Machine Learning Model
model= LogisticRegression(max_iter=1000)


In [ ]:
model.fit(X_train,Y_train)

In [ ]:
#Model Evaluation and Accuracy
X_train_prediction= model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)


In [ ]:
print('Accuracy Score on the training data', training_data_accuracy)

In [ ]:
X_test_prediction= model.predict(X_test)
test_data_accuracy= accuracy_score(Y_test,X_test_prediction)


In [ ]:
print('Accuracy Score on the test data', test_data_accuracy)